In [186]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import torch
import test_fn

In [187]:
import importlib
importlib.reload(test_fn)

<module 'test_fn' from 'C:\\Users\\ayeh21\\Documents\\HesBO\\baseline\\test_fn.py'>

# Draw sample inputs

In [188]:
n = 100
low_dim = 50
fn_dim = 200
fn = test_fn.Ackley()

In [189]:
x = torch.rand((n,low_dim))
A = torch.rand((low_dim, fn_dim))
X = x@A
y = fn.evaluate(X)

In [190]:
X[torch.topk(y, 3, largest = False).indices,:].shape

torch.Size([3, 200])

In [191]:
X.shape

torch.Size([100, 200])

# Algorithm 1: Shift subspace to best performing points

In [206]:
importlib.reload(test_fn)
fn = test_fn.Ackley()
x = torch.rand((100,low_dim))
A = torch.rand((low_dim, fn_dim))
X = x@A
y = fn.evaluate(X)

# Find best performing points (Alt: Find best performing and nearest neighbors)
best_performers_indices = torch.topk(y, 1, largest = False).indices
best_performers = X[best_performers_indices]

# Update
A = best_performers

In [207]:
A.shape

torch.Size([1, 200])

In [212]:
torch.concat([A, torch.rand((low_dim - 1, fn_dim))], dim = 0).shape

torch.Size([50, 200])

In [261]:
def learn_subspace_best_performers(fn, low_dim, fn_dim, num_best = 1, samples = 1000, max_iter = 1000):
    patience = max_patience
    best_found = 1e10
    A = torch.rand((low_dim, fn_dim))
    first_run = True
    
    for i in range(max_iter):
        # Evaluate
        x = torch.randn((samples,low_dim))
        X = x@A
        X = fn.restrict_domain(X)       
        y = fn.evaluate(X)

        # Find best performing points (Alt: Find best performing and nearest neighbors)
        best_performers_indices = torch.topk(y, num_best, largest = False).indices
        best_performers = X[best_performers_indices]
        
        # Standardize
        best_performers = best_performers - torch.mean(best_performers, dim = 1, keepdim = True)
        best_performers = best_performers/torch.std(best_performers, dim = 1, keepdim = True)

        # Update
        A = torch.concat([best_performers, torch.randn((low_dim - num_best, fn_dim))], dim = 0)

        if min(y) < best_found:
            best_found = min(y)
        else:
            patience -= 1

        if i%int(max_iter/10) == 0:
            print(f"Min/best found y is:{min(y):.2f}, {best_found:.2f}")

In [268]:
learn_subspace_best_performers(fn, low_dim = 50, fn_dim = 200, num_best = 4, samples = 100)

Min/best found y is:7.11, 7.11
Min/best found y is:14.88, 7.11
Min/best found y is:14.21, 7.11
Min/best found y is:14.89, 7.11
Min/best found y is:14.55, 7.11
Min/best found y is:14.69, 7.11
Min/best found y is:14.67, 7.11
Min/best found y is:14.64, 7.11
Min/best found y is:14.40, 7.11
Min/best found y is:14.60, 7.11


In [205]:
best_performers

tensor([[ 9.2315,  7.7709,  7.8833,  ...,  7.9377,  7.3055,  8.1079],
        [ 9.7218,  9.5276,  9.3146,  ...,  8.9890,  8.3138,  8.2052],
        [10.3954,  9.0493,  9.0542,  ...,  8.6739,  8.4123,  9.0580],
        ...,
        [12.8120, 12.3960, 12.1742,  ..., 10.6394, 11.1695, 10.3386],
        [12.3022, 12.2804, 12.8497,  ...,  9.6569,  9.5674,  9.9907],
        [12.9382, 12.6913, 12.3818,  ..., 10.8169, 10.4931, 10.9597]])

# Algorithm 2: Shift subspace to best performing point and nearest neighbors